In [1]:
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
import glob
from ketos.audio.audio_loader import  AudioFrameLoader, FrameStepper, audio_repres_dict
from ketos.neural_networks import load_model_file
from ketos.neural_networks.resnet import ResNetInterface
from ketos.neural_networks.dev_utils.detection import process, process_audio_loader, save_detections, merge_overlapping_detections

2022-02-06 17:41:05.850687: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-02-06 17:41:17.278669: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-06 17:41:17.282586: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-06 17:41:17.282672: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-06 17:41:17.283043: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cdr845.int.cedar.computecanada.ca): /proc/driver/nvidia/version does not exist
2022-02-06 17:41:17.285245: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network 

In [2]:
model='../../trained_models/kw_multi_detector_v03_3class.kt'
output_file_short_note='_v03_3class_' # add anything here to add in the name of the detections file! By default keep it an empty string
annot_file_path='../../annotations/test/' # Path where test annotations are stored
detection_file_path='../../model_detections/'
test_file_list='../../file_lists/test/'
audio_storage_root_path='/home/sadman/projects/ctb-ruthjoy/SRKW/'

num_segs=128
step_size=None
buffer=0.0
win_len=1
threshold=0.0
group=False
progress_bar=True
merge=False

save_detections_flag=True
save_all_performance_flag=False


In [3]:
# load the classifier and the spectrogram parameters
model, audio_repr = load_model_file(model, './tmp_folder', load_audio_repr=True)
spec_config = audio_repr[0]['spectrogram']
spec_config

{'duration': 5.0,
 'rate': 10000,
 'window': 0.051,
 'step': 0.01955,
 'freq_min': 0,
 'freq_max': 6000,
 'window_func': 'hamming',
 'normalize_wav': True,
 'type': 'MagSpectrogram',
 'transforms': [{'name': 'reduce_tonal_noise'},
  {'name': 'normalize', 'mean': 0.0, 'std': 1.0}]}

In [4]:
def process_batch(batch_data, batch_support_data, model, buffer=0, step=0.5, spec_dur=3.0, threshold=0.5, win_len=1, group=False):
    """ Runs one batch of (overlapping) spectrogram throught the classifier.

        Args:
            batch_data: numpy array
                An array with shape n,f,t,  where n is the number of spectrograms in the batch, t is the number of time bins and f the number of frequency bins.
            batch_support_data: numpy array
                An array of shape n x 2, where n is the batch size. The second dimension contains the filename and the start timestamp for each input in the batch
            model: ketos model
                The ketos trained classifier
            buffer: float
                Time (in seconds) to be added around the detection
            step: float
                The time interval(in seconds) between the starts of each contiguous input spectrogram.
                For example, a step=0.5 indicates that the first spectrogram starts at time 0.0s (from the beginning of the audio file), the second at 0.5s, etc.
            spec_dur: float
                The duration of each input spectrogram in seconds
            threshold: float or list of floats
                Minimum score value for a time step to be considered as a detection.
            win_len:int
                The windown length for the moving average. Must be an odd integer. The default value is 5.
            group:bool
                If False, return the filename, start, duration and scores for each spectrogram with score above the threshold. In this case, the duration will always be the duration of a single spectrogram.
                If True (default), average scores over(overlapping) spectrograms and group detections that are immediatelly next to each other. In this case, the score given for that detection will be the
                average score of all spectrograms comprising the detection event.

        Returns:
            batch_detections: list
                An array with all the detections in the batch. Each detection (first dimension) consists of the filename, start, duration and score.
                The start is given in seconds from the beginning of the file and the duration in seconds. 
                If a list of threshold values is specified, the returned object will be a list of arrays with len(batch_detections) = len(thresholds)         

    """
    thresholds = threshold if isinstance(threshold, list) else [threshold]

    probability_scores = model.run_on_batch(batch_data, return_raw_output=True)
    class_index = np.argmax(np.array(probability_scores), axis=1)
    # label_to_text_mapping={0: 'OTHER', 1: 'KW', 2: 'HB', 3: 'D'}

    if win_len == 1:
        scores = probability_scores[:,1]
    else:
        scores = compute_avg_score(probability_scores[:,1], win_len=win_len)
        
    if group == True:
        batch_detections = group_detections(scores, batch_support_data, buffer=buffer, step=step, spec_dur=spec_dur, threshold=thresholds) 

    else:
        batch_detections = []

        for thres in thresholds:
            threshold_indices = scores >= thres
            batch_det = np.vstack([batch_support_data[threshold_indices,0], batch_support_data[threshold_indices,1], np.repeat(spec_dur, sum(threshold_indices)), scores[threshold_indices]])
            if batch_det.shape[1] == 0:
                batch_det = []
            else:
                batch_det = [(batch_det.T[det_index][0], 
                              float(batch_det.T[det_index][1]), 
                              float(batch_det.T[det_index][2]), 
                              float(batch_det.T[det_index][3]), 
                              int(class_index[det_index])) for det_index in range(len(batch_det.T))]

            batch_detections.append(batch_det)

    if not isinstance(threshold, list): 
        batch_detections = batch_detections[0]
        
    return batch_detections

def process_audio_loader(audio_loader, model, batch_size=128, threshold=0.5, buffer=0, win_len=1, group=False, progress_bar=False, merge=False):
    """ Use an audio_loader object to compute spectrogram from the audio files and process them with the trained classifier.

        Args:
            audio_loader: a ketos.audio.audio_loader.AudioFrameLoader object
                An audio loader that computes spectrograms from the audio audio files as requested
            model: ketos model
                The ketos trained classifier
            batch_size:int
                The number of spectrogram to process at a time.
            threshold: float or list of floats
                Minimum score value for a time step to be considered as a detection.
            buffer: float
                Time (in seconds) to be added around the detection
            win_len:int
                The windown length for the moving average. Must be an odd integer. The default value is 5.   
            group:bool
                If False, return the filename, start, duration and scores for each spectrogram with score above the threshold. In this case, the duration will always be the duration of a single spectrogram.
                If True (default), average scores over(overlapping) spectrograms and group detections that are immediatelly next to each other. In this case, the score given for that detection will be the
                average score of all spectrograms comprising the detection event.
            progress_bar: bool
                Show progress bar.  
            merge: bool
                Apply :func:`merge_overlapping_detections` to the detections before they are returned. Default is False.

        Returns:
            detections: list
                List of detections.
                If a list of threshold values is specified, the returned object will be a list of lists with len(detections) = len(thresholds)         
    """
    assert isinstance(win_len, int) and win_len%2 == 1, 'win_len must be an odd integer'

    thresholds = threshold if isinstance(threshold, list) else [threshold]
        
    n_extend = int((win_len - 1) / 2)

    n_batches = audio_loader.num() // batch_size
    last_batch_size = batch_size + (audio_loader.num() % batch_size)

    if n_batches == 0: 
        batch_sizes = [audio_loader.num()]
    elif n_batches == 1:
        batch_sizes = [last_batch_size]
    else:
        batch_sizes = [batch_size + n_extend] + [batch_size + 2 * n_extend for _ in range(n_batches - 2)] + [last_batch_size + n_extend]

    detections = [[] for _ in range(len(thresholds))]
    specs_prev_batch = []
    duration = None
    step = 0

    for siz in tqdm(batch_sizes, disable = not progress_bar): 
        batch_data, batch_support_data = [], []

        # first, collect data from the last specs from previous batch, if any            
        for spec in specs_prev_batch: 
            batch_data.append(spec.data)
            support_data = (spec.filename, spec.offset)
            batch_support_data.append(support_data)
            duration = spec.duration()

        # then, load specs from present batch
        specs_prev_batch = []
        while len(batch_data) < siz:
            spec = next(audio_loader)
            batch_data.append(spec.data)
            support_data = (spec.filename, spec.offset)
            batch_support_data.append(support_data)
            if siz - len(batch_data) < 2 * n_extend: specs_prev_batch.append(spec) # store last few specs
            duration = spec.duration()

        if len(batch_support_data) >= 2:
            step = batch_support_data[1][1] - batch_support_data[0][1]

        if step <= 0: step = duration

        batch_support_data = np.array(batch_support_data)
        batch_data = np.array(batch_data)

        batch_detections = process_batch(batch_data=batch_data, batch_support_data=batch_support_data, model=model, threshold=thresholds, 
                                        buffer=buffer, step=step, spec_dur=duration, win_len=win_len, group=group)

        for i in range(len(thresholds)):
            if len(batch_detections[i]) > 0: detections[i] += batch_detections[i]

    if merge:
        for i in range(len(thresholds)):
            detections[i] = merge_overlapping_detections(detections[i])

    if not isinstance(threshold, list): 
        detections = detections[0]

    return detections

In [5]:
def save_detections(detections, save_to):
    """ Save the detections to a csv file

        Args:
            detections: numpy.array
                List of detections
            save_to:string
                The path to the .csv file where the detections will be saved.
                Example: "/home/user/detections.csv"
    """
    if len(detections) == 0: return

    a = np.array(detections)
    df = pd.DataFrame({'filename':a[:,0], 'start':a[:,1], 'duration':a[:,2], 'predicted_label':a[:,4]})
    include_header = not os.path.exists(save_to)
    df.to_csv(save_to, mode='a', index=False, header=include_header)
    return df

In [6]:
def remove_missing_files(dataframe):
    """ Save the detections to a csv file

        Args:
            dataframe: pandas.DataFrame
                List of files for testing a detector
                
        Returns: 
            : pandas.DataFrame
                Filtered dataframe after removing the files that do not exist
    """
    for index, row in dataframe.iterrows():
        if(os.path.isfile(row['filename'])==False):
            print(row['filename'], " not found")
            dataframe = dataframe.drop([index])
    return dataframe
    

In [7]:
all_results_df=pd.DataFrame(columns=['dataset', 
                                     'accuracy', 
                                     'precision', 
                                     'recall', 
                                     'false_positive_rate', 
                                     'confusion_matrix'])

# load each test file and do all the calculations on them
if test_file_list is not None:
    for file_path in glob.glob(test_file_list+'*.csv'):
        file_list = pd.read_csv(file_path)
        
        # appending audio folder location to each file name
        file_list['filename'] = audio_storage_root_path + file_list['filename'].astype(str)

        print("Number of files BEFORE removing missing files:", len(file_list))
        # Make sure each file actually exists in the cedar, if not, then remove from dataframe
        file_list = remove_missing_files(file_list)
        print("Number of files AFTER removing missing files:", len(file_list))

        file_list = list(file_list['filename'])
        
        # extract the audio folder path
        last_index_of_slash=file_list[0].rindex('/')
        audio_folder=file_list[0][0:last_index_of_slash]
        
        # extract the file name from the folder path in the format of "filename.csv"
        file_name=file_path[file_path.rindex('/')+1:len(file_path)]
        print("file_name", file_name)
        
        # initialize the audio loader
        audio_loader = AudioFrameLoader(frame=spec_config['duration'], 
                                        step=step_size, 
                                        path=audio_folder, 
                                        filename=file_list, 
                                        repres=spec_config)

        # get the detection scores
        detections = process_audio_loader(audio_loader, 
                                          model=model, 
                                          batch_size=num_segs, 
                                          buffer=buffer, 
                                          threshold=threshold, 
                                          group=group, 
                                          win_len=win_len, 
                                          progress_bar=progress_bar,
                                          merge=merge)
            
        if save_detections_flag == True:
            # get the output file name
            output=detection_file_path+"detections_"+file_name
                
            # save the each detections on test dataset
            if os.path.isfile(output): os.remove(output) #remove, if already exists
            print(f'{len(detections)} detections saved to {output}')
            detections_df=save_detections(detections=detections, save_to=output)
            

Number of files BEFORE removing missing files: 22
Number of files AFTER removing missing files: 22
file_name orcasound.csv


  0%|          | 0/4 [00:00<?, ?it/s]2022-02-06 17:42:35.526901: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-06 17:42:35.556166: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2095105000 Hz
100%|██████████| 4/4 [02:15<00:00, 33.97s/it]


634 detections saved to ../../model_detections/detections_orcasound.csv
Number of files BEFORE removing missing files: 133
Number of files AFTER removing missing files: 133
file_name superpod_lime_kiln.csv


100%|██████████| 12/12 [11:38<00:00, 58.24s/it]


1646 detections saved to ../../model_detections/detections_superpod_lime_kiln.csv
Number of files BEFORE removing missing files: 42
Number of files AFTER removing missing files: 42
file_name jasco_roberts_bank.csv


...

44826 detections saved to ../../model_detections/detections_onc_barkley_canyon_test_multiclass.csv
Number of files BEFORE removing missing files: 5
Number of files AFTER removing missing files: 5
file_name jasco_boundary_pass.csv


100%|██████████| 14/14 [07:37<00:00, 32.64s/it]


1800 detections saved to ../../model_detections/detections_jasco_boundary_pass.csv
Number of files BEFORE removing missing files: 73
Number of files AFTER removing missing files: 73
file_name onc_barkley_canyon.csv


100%|██████████| 34/34 [14:52<00:00, 26.26s/it]


4380 detections saved to ../../model_detections/detections_onc_barkley_canyon.csv
